In [106]:
from datetime import timedelta

import pandas as pd


ORDERS_PATH='D:/DA project/sales-analytics-project/data/clean/orders_clean.csv'
orders = pd.read_csv(ORDERS_PATH, parse_dates=["order_purchase_timestamp"])

In [107]:
print(orders["order_purchase_timestamp"].min())
print(orders["order_purchase_timestamp"].max())

2016-09-04 21:15:19
2018-10-17 17:30:18


In [116]:
# Anchor to last full date
end_date = pd.Timestamp("2018-08-23")

# AFTER window (28 days, inclusive)
after_end = end_date
after_start = after_end - pd.Timedelta(days=27)

# BEFORE window (previous 28 days, inclusive)
before_end = after_start - pd.Timedelta(days=1)
before_start = before_end - pd.Timedelta(days=27)

after_start, after_end, before_start, before_end


(Timestamp('2018-07-27 00:00:00'),
 Timestamp('2018-08-23 00:00:00'),
 Timestamp('2018-06-29 00:00:00'),
 Timestamp('2018-07-26 00:00:00'))

In [123]:
(after_end - after_start).days + 1, (before_end - before_start).days + 1


(28, 28)

In [71]:
''' end_date = orders["order_purchase_timestamp"].max().normalize()
before_end = end_date
before_start= before_end - pd.Timedelta(days=27)
previous_end= before_start - pd.Timedelta(days=1)
previous_start= previous_end - pd.Timedelta(days=27)



(previous_end-previous_start ).days+1, (end_date-before_start).days+1 '''

(28, 28)

In [117]:
orders["order_date"] = orders["order_purchase_timestamp"].dt.normalize()


In [118]:
def label_period(ts):
    if before_start <= ts <= before_end:
        return "before"
    elif after_start <= ts <= after_end:
        return "after"
    else:
        return None

orders["period"] = orders["order_date"].apply(label_period)


In [119]:
orders_period = orders.dropna(subset=["period"])

orders_period["period"].value_counts()


period
after     7304
before    5427
Name: count, dtype: int64

In [130]:
conversion_counts = (
    orders_period
    .groupby("period")["order_id"]
    .nunique()
)

conversion_counts


period
after     7304
before    5427
Name: order_id, dtype: int64

In [132]:
import numpy as np
from scipy.stats import poisson

orders_before = 5427
orders_after  = 7304

days_before = 28
days_after  = 28

rate_before = orders_before / days_before
rate_after  = orders_after / days_after

rate_before, rate_after


(193.82142857142858, 260.85714285714283)

In [134]:
# rate ratio
rate_ratio = rate_after / rate_before

# Standard error of log(rate ratio)
se = np.sqrt(1/orders_after + 1/orders_before)

# z-statistic
z_stat = np.log(rate_ratio) / se

# two-sided p-value
from scipy.stats import norm
p_value = 2 * (1 - norm.cdf(abs(z_stat)))

print(z_stat, p_value)


16.574392053939718 0.0


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 1.2 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.1 MB 1.2 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 999.4 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.1 MB/s eta 0:00:02
   ----------------------------- ---------- 1.6/2.1 MB 1.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.3 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [greenlet]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   ---------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
